# Hamiltonian Matrix Construction

### Imports

In [1]:
from pyscf import gto, scf
import scipy.sparse as sp
import numpy as np
from scipy.sparse.linalg import eigsh

### Tries to construct the hamiltonian matrix

In [5]:
from pyscf import gto, scf
import numpy as np

# Define the molecule (water in this case)
mol = gto.M(atom=[
    ['O', (0.0, 0.0, 0.0)],          # Oxygen at origin
    ['H', (0.0, -0.757, 0.587)],     # Hydrogen 1
    ['H', (0.0, 0.757, 0.587)]       # Hydrogen 2
], basis='cc-pVTZ')  # Using a larger basis set

# Perform Hartree-Fock calculation
mf = scf.RHF(mol)
mf.kernel()

# Get MO coefficients
mo_coeff = mf.mo_coeff

# Get one-electron and two-electron integrals
h1 = mf.get_hcore(mol)
g2 = mol.intor('int2e', aosym='s1')

# Transform g2 to MO basis
g2_mo = np.einsum('pqrs,pi,qj,rk,sl->ijkl', g2, mo_coeff, mo_coeff, mo_coeff, mo_coeff)

# Construct the Hamiltonian matrix in MO basis
num_orbitals = mo_coeff.shape[1]
H = np.zeros((num_orbitals, num_orbitals))

# Add one-electron integrals
for i in range(num_orbitals):
    for j in range(num_orbitals):
        H[i, j] += h1[i, j]

# Add two-electron integrals
for i in range(num_orbitals):
    for j in range(num_orbitals):
        for k in range(num_orbitals):
            for l in range(num_orbitals):
                H[i, j] += 0.5 * g2_mo[i, j, k, l] * ((k == l) - 0.5 * (i == l) * (j == k))

# Diagonalize the Hamiltonian matrix
eigenvalues, eigenvectors = np.linalg.eigh(H)

# eigenvalues contains the energy levels
print("Eigenvalues:", eigenvalues)


converged SCF energy = -76.057114083119


#### See the true lowest eigenvalue

In [3]:

# Assuming large_H_spin_sparse_csr is your large sparse Hamiltonian matrix
# Find the smallest eigenvalue
# 'which='SA'' means to find the smallest algebraic eigenvalue
eigenvalues, eigenvectors = eigsh(large_H_spin_sparse_csr, k=1, which='SA')

# Extract the smallest eigenvalue
smallest_eigenvalue = eigenvalues[0]
print("Smallest eigenvalue:", smallest_eigenvalue)

Smallest eigenvalue: -37308.77372912153


#### Check the size of the matrix

In [4]:
large_H_spin_sparse_csr.shape

(16384, 16384)